# Interacting with the ESCALATE REST API

This is a tutorail for using the ESCALATE REST API. 

It presumes only knowledge of basic python. 

Currently you need a local instance of ESCALATE running, but this will be replaced with a public facing demo instance in the future.

In [1]:
import json
from pprint import pprint

import requests  # requests library will send and receive data from the escalate server
from requests.api import post

from IPython.display import JSON  # a handy way to interact with JSON jupyter notebooks (only works in JupyterLab)

## Quick intro to REST APIs

(skip this if familair with REST APIs)

All you need to know about REST is that:
* It is a protocol for exchanging data between a client (e.g. me) and a server (e.g. ESCALATE)
* Data formats are human *and* machine readable (XML or JSON)
* There are two main HTTP 'verbs' or functions: 
  1. GET data from the server
  2. POST data to the server  
     (there are others, e.g. PUT and PATCH, but we won't use these in this tutorial)
* The python `requests` library implements the HTTP verbs for interacting with servers, including with REST APIs

### Example: GET from the PubChem API

* [Pubchem](https://pubchem.ncbi.nlm.nih.gov/) is a great way to get chemical informatics data from a vast repository
   - e.g. just search for any compound above
* If I wanted to get a lot of data from PubChem I would use the REST API

In [2]:
PubChemIDs = {
    'Methane': '297',
    'Benzene': '241',
    'Ethylammonium Iodide': '11116533'
}

The URL below is the compound 'API endpoint' which I can send a request to for properties of compounds given their PubChem CIDs

In [3]:
response = requests.get(('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound'          # Compound endpoint
                         f'/cid/{",".join(PubChemIDs.values())}/'                      # Compound IDs from above as a comma separated list
                         'property/MolecularFormula,MolecularWeight,CanonicalSMILES/'  # the properties I want back
                         'JSON'))                                                      # the format I want the data in (XML and CSV are also available)

This call to `requests.get` returns a `requests.response` object. It contains all data returned from the server. If that includes JSON, we can parse it into a python dictionary:

In [4]:
pubchem_response_json = response.json() 
pubchem_response_json

{'PropertyTable': {'Properties': [{'CID': 297,
    'MolecularFormula': 'CH4',
    'MolecularWeight': 16.043,
    'CanonicalSMILES': 'C'},
   {'CID': 241,
    'MolecularFormula': 'C6H6',
    'MolecularWeight': 78.11,
    'CanonicalSMILES': 'C1=CC=CC=C1'},
   {'CID': 11116533,
    'MolecularFormula': 'C2H8IN',
    'MolecularWeight': 173,
    'CanonicalSMILES': 'CCN.I'}]}}

And then we can use it for whatever we want!

## The ESCALATE REST API

So REST is a great way to send data back and forth between a program and a server. 

Lets now see how we can use the ESCALATE API to: 
1. Model materials and material properties
2. Define experimental protocols
3. Send these experimental programs to a remote laboratory for execution
4. Retrieve data from this remote laboratory to close the loop

## Interactive List of all available ESCALATE endpoints

Simply navigating to http://localhost:8000/api will show you a browsable list of all available API endpoints. 

We can also view that list programmatically: 

In [5]:
base_url = 'http://localhost:8000/api'  # local dev server
response = requests.get(base_url)
response.json()

{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/actiondef/',
 'actionparameter': 'http://localhost:8000/api/actionparameter/',
 'actionparameterdefassign': 'http://localhost:8000/api/actionparameterdefassign/',
 'actor': 'http://localhost:8000/api/actor/',
 'billofmaterials': 'http://localhost:8000/api/billofmaterials/',
 'bomcompositematerial': 'http://localhost:8000/api/bomcompositematerial/',
 'bommaterial': 'http://localhost:8000/api/bommaterial/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculationdef/',
 'compositematerial': 'http://localhost:8000/api/compositematerial/',
 'compositematerialproperty': 'http://localhost:8000/api/compositematerialproperty/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditioncalculationdefassign': 'http://localhost:8000/api/conditioncalculationdefassign/',
 'conditiondef': 'http://localhost:8000/api/conditiondef/',
 'experiment': '

### Logging into ESCALATE


To be able to view and post to all endpoints, I will create a log in session that is managed by a token. 

In [6]:
# demo login credentials
login_data = {
    'username': 'mtynes',
    'password': 'hello1world2'
}

r_login = requests.post(f'{base_url}/login', data=login_data)
token = r_login.json()['token']
token

'3ceb9d93d1e1ede85bee290375f4b06abafa878b'

In [7]:
token_header = {'Authorization': f'Token {token}'}
content_type_header = {'content-type': 'application/json'}

This token will allow me to validate my identity for this session

### Simple helper functions for GET/POST

These functions will do some minimal URL generation and JSON response parsing on top of what is done by the `requests` library

In [8]:
def post_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """POST `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.post(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        print('POST: OK, returning new resource dict')
        return r.json()
    print('POST: FAILED, returning response object')
    return r

def get_data(endpoint, data={}, headers={**token_header}):
    """Make GET request with `data` to `endpoint` in ESCALATE API using `headers`
    
    return: (dict|list|requests.Response), bool
    """
    r = requests.get(f'{base_url}/{endpoint}/', params=data, headers=headers)
    print(r)
    if r.ok: 
        print('GET: OK')
        
        resp_json = r.json()        
        
        # handle cases: one vs many results
        if resp_json.get('count') is None: # edge case: template edit
            return r.json()
        elif resp_json.get('count') == 1: 
            print('Found one resource, returning dict')
            return resp_json['results'][0]
        elif resp_json.get('count') >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
    print('GET: FAILED, returning response object')
    return r

### GET all of materials defined in ESCALATE

In [9]:
r = get_data(endpoint='material')

<Response [200]>
GET: OK
Found 316 resources, returning list of dicts)


316 materials would probably take up too much space in this notebook, lets look at the first one.

In [10]:
r[0]

{'url': 'http://localhost:8000/api/material/35d2d912-e5d1-4e49-9900-ef58772f3ae7/',
 'uuid': '35d2d912-e5d1-4e49-9900-ef58772f3ae7',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Formic Acid',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-04-14T16:06:51.349834Z',
 'mod_date': '2021-04-14T16:07:04.174391Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/df4011a8-c950-47d4-aa75-3dea85798756/',
 'material_types': ['http://localhost:8000/api/materialtype/933d6976-cc28-41bd-bf5e-b05ce10c5e89/',
  'http://localhost:8000/api/materialtype/0d8e3f80-ba8e-459e-a18b-3f1f84494f78/',
  'http://localhost:8000/api/materialtype/3334ac7c-20ae-4ea3-8fe4-3031226c62ea/',
  'http://localhost:8000/api/materialtype/9f769f9f-b3d9-4d0f-9588-e9694edc5cb3/'],
 'property': []}

These are the fields available for materials. Notably we can associate a material with arbitrary properties and material types.

### POST a material property

We can define arbitrary properties in ESCALATE and associate them with materials

In [11]:
get_data('propertydef', {'fields': ['description']})

<Response [200]>
GET: OK
Found 16 resources, returning list of dicts)


[{'description': 'molecular-weight'},
 {'description': 'concentration'},
 {'description': 'plate well volume {min, max}'},
 {'description': 'plate well robot order'},
 {'description': 'plate well location'},
 {'description': 'plate well count'},
 {'description': 'manufacturer'},
 {'description': 'concentration_molarity'},
 {'description': 'concentration_rad'},
 {'description': 'Functional group'},
 {'description': 'Resin Class'},
 {'description': 'moisture % {min, max}'},
 {'description': 'cross-linkage %'},
 {'description': 'capacity'},
 {'description': 'mesh {min, max}'},
 {'description': 'particle-size {min, max}'}]

These are the property definitions we use, we are free to define more

Lets take a look at a specific one

In [12]:
mw_property_def = get_data('propertydef', 
                           {'description': 'molecular-weight', 
                            'fields': ['url', 'uuid', 'description', 'val_unit']})
mw_property_def

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/propertydef/d499f888-7ef8-452a-9c55-e7e4a49bcede/',
 'uuid': 'd499f888-7ef8-452a-9c55-e7e4a49bcede',
 'description': 'molecular-weight',
 'val_unit': 'g/mol'}

Now we can associate an instance of this property with a material, say EthNH3I

In [13]:
ethylammonium_iodide = get_data('material', 
                                {'description': 'Ethylammonium Iodide',
                                 'fields':['url', 'uuid', 'description', 'property']})

<Response [200]>
GET: OK
Found one resource, returning dict


Get the property from pubchem

In [14]:
response = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/'+\
                        f'compound/cid/{PubChemIDs.get("Ethylammonium Iodide")}/'+\
                        'property/MolecularFormula,MolecularWeight,CanonicalSMILES/JSON')
pubchem_json = response.json()
eth_mw = pubchem_json['PropertyTable']['Properties'][0]['MolecularWeight']
eth_mw

173

In [15]:
r = post_data('materialproperty',
                  {'material': ethylammonium_iodide['url'],
                   'property_def': mw_property_def['url'],
                   'value': {"value": f"{eth_mw}",
                             "type": "text",
                             "unit": "g/mol"
                             }
                  }
                 )

<Response [201]>
POST: OK, returning new resource dict


And we've stored it!

In [16]:
r

{'url': 'http://localhost:8000/api/materialproperty/36f9068d-0763-42e1-9c88-b520d2e91c48/',
 'uuid': '36f9068d-0763-42e1-9c88-b520d2e91c48',
 'edocs': [],
 'tags': [],
 'notes': [],
 'material_class': '',
 'description': None,
 'property_description': None,
 'property_short_description': None,
 'value': {'value': '173', 'unit': 'g/mol', 'type': 'text'},
 'actor_description': 'Mike Tynes',
 'status_description': None,
 'add_date': '2021-04-14T20:40:04.343257Z',
 'mod_date': '2021-04-14T20:40:04.343273Z',
 'material': 'http://localhost:8000/api/material/538b82e3-6b0d-46b9-880c-7f278d6137e1/',
 'property': None,
 'property_def': 'http://localhost:8000/api/propertydef/d499f888-7ef8-452a-9c55-e7e4a49bcede/',
 'actor': 'http://localhost:8000/api/actor/a1af66f1-0944-4233-9d08-e76931848943/',
 'status': None}

In [17]:
get_data('material', 
         {'description':'Ethylammonium Iodide'})

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/538b82e3-6b0d-46b9-880c-7f278d6137e1/',
 'uuid': '538b82e3-6b0d-46b9-880c-7f278d6137e1',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Ethylammonium Iodide',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-04-14T16:06:51.349834Z',
 'mod_date': '2021-04-14T16:07:04.174391Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/df4011a8-c950-47d4-aa75-3dea85798756/',
 'material_types': ['http://localhost:8000/api/materialtype/0d8e3f80-ba8e-459e-a18b-3f1f84494f78/',
  'http://localhost:8000/api/materialtype/92252270-a0dc-4f38-858d-862faccd4156/',
  'http://localhost:8000/api/materialtype/3334ac7c-20ae-4ea3-8fe4-3031226c62ea/'],
 'property': ['http://localhost:8000/api/property/8ed88019-486e-483f-8e0d-77e7ced943d2/',
  'http://localhost:8000/api/property/1fad1670-cd71-474d-b66a-8397e53854f0/']}


In practice we can use this functionality to store properties from any experiment or calculation. 

We can also store metadata about where these values came from (example to come in a further tutorial on tags, notes, edocs, calculations).

## Action definitions

* Just as we are free to define properties, we are free to define actions
* The ones below are used in our protocols

In [18]:
r = get_data('actiondef', {'fields': ['description', 'uuid', 'url']})
r

<Response [200]>
GET: OK
Found 6 resources, returning list of dicts)


[{'url': 'http://localhost:8000/api/actiondef/75bdfa9f-ccb8-4ccb-b66d-e4ff9b3c8d91/',
  'uuid': '75bdfa9f-ccb8-4ccb-b66d-e4ff9b3c8d91',
  'description': 'dispense'},
 {'url': 'http://localhost:8000/api/actiondef/b4dcd6f7-c620-406f-8579-7d3b252c815c/',
  'uuid': 'b4dcd6f7-c620-406f-8579-7d3b252c815c',
  'description': 'heat_stir'},
 {'url': 'http://localhost:8000/api/actiondef/3f1e76ce-513f-43c2-b563-5d4fb72234f2/',
  'uuid': '3f1e76ce-513f-43c2-b563-5d4fb72234f2',
  'description': 'heat'},
 {'url': 'http://localhost:8000/api/actiondef/a8a01174-4e47-4e6d-bedf-a447d17fb5b2/',
  'uuid': 'a8a01174-4e47-4e6d-bedf-a447d17fb5b2',
  'description': 'start_node'},
 {'url': 'http://localhost:8000/api/actiondef/1de3537b-0e02-4762-b7c9-3f8e3c4b30fd/',
  'uuid': '1de3537b-0e02-4762-b7c9-3f8e3c4b30fd',
  'description': 'end_node'},
 {'url': 'http://localhost:8000/api/actiondef/09a5eec5-2b24-4941-a494-0cbfabf8552b/',
  'uuid': '09a5eec5-2b24-4941-a494-0cbfabf8552b',
  'description': 'dispense_solid'}]

In [19]:
get_data('actiondef', {'description': 'dispense', 
                               'fields': ['descrition', 'url', 'parameter_def'],
                                'expand': 'parameter_def', 
                       })

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/actiondef/75bdfa9f-ccb8-4ccb-b66d-e4ff9b3c8d91/',
 'parameter_def': [{'url': 'http://localhost:8000/api/parameterdef/802c813b-05e4-41b3-b8cb-eadb3f8a7a5e/',
   'uuid': '802c813b-05e4-41b3-b8cb-eadb3f8a7a5e',
   'edocs': [],
   'tags': [],
   'notes': [],
   'description': 'volume',
   'default_val': {'value': 0.0, 'unit': 'mL', 'type': 'num'},
   'required': True,
   'actor_description': 'Mike Tynes',
   'status_description': 'dev_test',
   'add_date': '2021-04-14T16:07:00.153430Z',
   'mod_date': '2021-04-14T16:07:00.153430Z',
   'val_type': 'http://localhost:8000/api/typedef/4a301a8d-ede8-466d-b48b-9b60304ddf49/',
   'actor': 'http://localhost:8000/api/actor/a1af66f1-0944-4233-9d08-e76931848943/',
   'status': 'http://localhost:8000/api/status/857d83df-9bc0-4d3b-a789-7ef2772d8cf9/'}]}

## Combining actions and materials into an experiment template

In [20]:
perovskite_template = get_data('experimenttemplate', 
                               {'description': 'test_wf_1', 
                               'expand':'workflow'})
JSON(perovskite_template)

<Response [200]>
GET: OK
Found one resource, returning dict


<IPython.core.display.JSON object>

We see in this JSON Structure three main nested fields: 
* Bill of materials <-- the initial materials for the perovskite workflow
* Workflow <-- the set of actions that combine these materials into perovskite crystal trials

## Bill of Materials

The initial materials: think list of materials in methods section of paper

In [21]:
bill_of_materials = perovskite_template['bill_of_materials'][0]['uuid']

In [22]:
bill_of_materials

'69793df3-e9c9-4124-883e-d9bc04010872'

In [23]:
get_data('bommaterial', {'bom': bill_of_materials, 
                        'fields':['description']})

<Response [200]>
GET: OK
Found 5 resources, returning list of dicts)


[{'description': 'Plate'},
 {'description': 'Stock A'},
 {'description': 'Stock B'},
 {'description': 'Acid'},
 {'description': 'Solvent'}]

In [24]:
get_data('compositematerial', {'composite_description__startswith':'Stock', 
                               'fields': ['composite_description',
                                          'component_description']})

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'composite_description': 'Stock A',
  'component_description': 'Lead Diiodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock B',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock B',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock FAH',
  'component_description': 'Formic Acid'},
 {'composite_description': 'Stock FAH', 'component_description': 'Water'}]

We could drill further to get the concentrations, properties, etc

### Workflows: Logical groups of actions

In [25]:
[wf['description'] for wf in perovskite_template['workflow']]

['Dispense Solvent',
 'Dispense Stock A',
 'Dispense Stock B',
 'Dispense Acid Vol 1',
 'Dispense Acid Vol 2']

Each of these contains a set of parameters that we can edit

### Creating a new workflow from a template

In [26]:
editable_template = get_data(
    f'experimenttemplate/{perovskite_template["uuid"]}/create'
)

<Response [200]>
GET: OK


In [27]:
editable_template

{'experiment_name': '',
 'material_parameters': [{'material_name': 'Acid',
   'value': 'http://localhost:8000/api/bommaterial/092d5b72-8121-4b4c-b09f-b6932c7feec1/'},
  {'material_name': 'Solvent',
   'value': 'http://localhost:8000/api/bommaterial/ee69cab2-815a-4ff3-9b93-b6929f8aa322/'},
  {'material_name': 'Stock A',
   'value': 'http://localhost:8000/api/bommaterial/d39996c8-3c96-4bb6-a86c-3130e71e9453/'},
  {'material_name': 'Stock B',
   'value': 'http://localhost:8000/api/bommaterial/756703e1-120f-462a-981a-05210a695957/'},
  {'material_name': 'Plate',
   'value': 'http://localhost:8000/api/bommaterial/2ac83f3f-80cf-4805-a532-25cc9d430e9c/'}],
 'experiment_parameters_1': [],
 'experiment_parameters_2': [{'object_description': 'Dispense Solvent',
   'parameter_def_description': 'volume',
   'value': {'value': 1.0, 'unit': 'mL', 'type': 'num'}},
  {'object_description': 'Dispense Solvent',
   'parameter_def_description': 'volume',
   'value': {'value': 2.0, 'unit': 'mL', 'type': 'n

Can also give arrays of values over 96 well plates

Suppose I edit this json then I can post the new template to the server

In [28]:
resp = post_data(
    f'experimenttemplate/{perovskite_template["uuid"]}/create',
    editable_template
)

<Response [500]>
POST: FAILED, returning response object


#### Todo: quick fix of this error

This experiment then appears in the experiment queue. The experimentalist is notified, can download relevant robot input and upload observed values through forms

### TODO
* upload/download human-rater defined crystal quality forms 
* demo upload/download of raw vial images
* demo running Shekar's interactive plots on the data from above